# importing the modules Numpy, Pandas, Matplotlib, Sklearn and  DWT module.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/mitbih-database"))
# annoation, mitbih-database/mitbih_database


# Any results you write to the current directory are saved as output.

from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
#import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression , Ridge
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression  # for Logistic Regression algorithm

from sklearn import metrics #for checking the model accuracy

from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm

from sklearn.ensemble import RandomForestClassifier # A combine model of many decision t

from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Module for discrete wavelet transform
import pywt

from sklearn.externals import joblib

# Randomly selecting files for training and testing.

In [ ]:
lst1=[i for i in range(100,125)]
a=[i+100 for i in [10,20]]
for i in a:
    lst1.remove(i)
lst2=[i for i in range(200,235)]
a=[i+200 for i in [4,6,11,16,18,24,25,26,27,29]]
for i in a:
    lst2.remove(i)
lst1+=lst2
lst=np.array(lst1)
np.random.shuffle(lst)
train_lst,test_lst=lst[:43],lst[43:]
for i in test_lst:
    print(i)

# filtering function:
**Inputs:**

**x -> df['ML']**

**Outputs:**

**high (arr) -> Bandpass output of the raw (including noise)signal.**

In [ ]:
def filtering(x):
    ##--LOW PASS FILTER
    x=list(x)
    low=[0,0]
    x=12*[0]+x
    for i,j in zip(range(12,len(x)),range(2,len(x)+2)):
        low.append(2*low[j-1]-low[j-2]+x[i]-2*x[i-6]+x[i-12])
    low=low[2:]
#     print(len(low),len(x))
    for i,j in enumerate(low):
        low[i]/=35
    x=x[12:]
#     plt.plot(range(1000),x[:1000],c='r',label='MLII')
#     plt.plot(range(1000),low[:1000],c='b',label='Low pass')
#     plt.legend()
#     plt.show()

    ##--HIGH PASS FILTER
    high=[0]
    low=32*[0]+low
    for i,j in zip(range(32,len(low)),range(1,len(low)+1)):
        high.append(32*low[i-16]-low[i]+low[i-32]-high[j-1])
    low=low[32:]
    high=high[1:]
#     print(len(low),len(high))
    for i,j in enumerate(high):
        high[i]/=15
#     plt.plot(range(1000),high[:1000],c='r',label='IIR Filter')
#     plt.plot(range(1000),low[:1000],c='b',label='Low pass')
#     plt.legend()
#     plt.show()

    return high

# interpolation function
**Using the band pass filter output and index of QRS complex, we are downsampling the points between them to 100 points and returning the list of all these points**

In [ ]:
def interpolation(ind,high):
    total_peaks=[]
    print(len(ind))
    for i,j in zip(ind[:-1] , ind[1:]):
        diff=(j-i)/100
        val=i
        arr=[]
        for k in range(100):
            val+=diff
            arr.append(high[round(val)])
        total_peaks.append(arr)
#     print(len(total_peaks),len(total_peaks[1]))
    return total_peaks

# DWT function
**Using the result of the reduction_testing() we are applying DWT (Discrete Wavelet Transform) on sets of 100 points and storing the coefficients of that in list and returning it.**

**These list of coefficient will act as feature vector for the heart beat.**

In [ ]:
def wavelet_transform(total_peaks):
    for i in range(len(total_peaks)):
        cA,cD= pywt.dwt(total_peaks[i], 'db1')
        cA.shape=(1,50)
        cD.shape=(1,50)
        if i==0:
            CA=cA.copy()
            CD=cD.copy()
        else:
            CD=np.concatenate((CD,cD))
            CA=np.concatenate((CA,cA))
#     print(CA.shape)
    return CA,CD

# creating_dataframe function:
**The CA and CD coefficients returned will be converted into pandas dataframe and it is returned.**

In [ ]:
def creating_dataframe(CA):
    train_CA=pd.DataFrame(CA,columns=['f{}'.format(i) for i in range(50)])
    return train_CA

# Creating the dataset

In [ ]:
def create_dataset():
    dataset=pd.DataFrame(columns=['f{}'.format(i) for i in range(50)]+['Sample#']+['Type'])
    for i in train_lst:
#         Reading the files
        print('{} is processing'.format(i))
        df=pd.read_csv("../input/mitbih-database/mitbih_database/{}.csv".format(i))
        if i in [102,104]:
            df=df[["'sample #'","'V5'"]]
            df.rename(columns={"'V5'":"ML"},inplace=True)
        else:
            df=df[["'sample #'","'MLII'"]]
            df.rename(columns={"'MLII'":"ML"},inplace=True)
        df["ML"]=(df["ML"]-1024)*0.005
        an=pd.read_csv("../input/annoation/{}annotations.csv".format(i))
        an=an[['Time','Sample#','Type']]
#         Creating the Labels ( Category)
        an['Type'].replace(to_replace=['N', 'P', 'f', 'p', 'Q', '|', '+', 's', 't', '~', 'L', 'R','A', 'a','x', 'J', 'S','V', 'F','e', 'j', 'n', 'E','[',']','!'],value=[1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,3,3,4,4,4,4,5,5,5], inplace=True)
        print(i,an['Type'].unique())
#         Preprocessing
        bandpass_output=filtering(df['ML'])
#         Interpolation
        total_peaks=interpolation(an['Sample#'],bandpass_output)
#         Discrete Wavelet Transform
        CA,CD=wavelet_transform(total_peaks)
        print(i,CA.shape,len(an['Type']))
        peak2peak=an['Sample#'].diff()
        peak2peak=peak2peak[1:]
#         Creating the Dataframe
        DS=creating_dataframe(CA)
        DS=pd.concat([DS , peak2peak] , axis =1)
        DS=pd.concat([DS,an['Type'][1:]],axis=1)
#         Adding the dataframe to dataset
        dataset=pd.concat([dataset,DS],axis=0)
    
    dataset.dropna(inplace=True)
    try:
        dataset=dataset[dataset['Type']!='"']
        dataset=dataset[dataset['Type']!='/']
    except:
        pass
    dataset['Type']=dataset['Type'].astype('int32')
    
    return dataset
ds=create_dataset()

In [ ]:
# randomly sampling 20000 datapoints of category 1
print(ds['Type'].unique())
print(ds['Type'].value_counts())
ds_t1=ds[ds['Type']==1]
print(len(ds_t1))
ds_t2=ds[(ds['Type']==2)|(ds['Type']==3)|(ds['Type']==4)|(ds['Type']==5)]
print(len(ds_t2))
print(len(ds))
ds_t1_sampled=ds_t1.sample(20000)
ds=pd.concat([ds_t1_sampled,ds_t2],axis=0)
print(ds['Type'].value_counts())

# Saving dataset

In [ ]:
ds.to_csv('dataset.csv',index=False)
df=pd.read_csv('dataset.csv')
df.shape

# Training the dataset using SVM

In [ ]:
print('phase1')
X_train, X_test, Y_train, Y_test = train_test_split(ds.iloc[:,0:51],ds.iloc[:,51] , test_size = 0.25)

print('phase2')
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)
print(type(X_train))
print(X_train.describe())

# Random Forest

In [ ]:
#Random Forest
clf2 = RandomForestClassifier(n_estimators=100 ,random_state=1)
clf2.fit(X_train, Y_train)
pre = clf2.predict(X_test)

#Saving model
filename = 'random_forest2.sav'
joblib.dump(clf2, filename)

#Using Current Classfier
print('phase4')
pre=clf2.predict(X_test)

#Printing the accuracy score
print('phase5')
print(accuracy_score(Y_test,pre))
print(confusion_matrix(Y_test, pre))

# Predictions

# Decision Tree

In [ ]:
#decison tree classifier
clf3 = DecisionTreeClassifier()
clf3=clf3.fit(X_train,Y_train)
pre=clf3.predict(X_test)

#Saving model
filename = 'Decison_Tree2.sav'
joblib.dump(clf3, filename)

#Using Current Classfier
print('phase4')
pre=clf3.predict(X_test)

#Printing the accuracy score
print('phase5')
print(accuracy_score(Y_test,pre))
print(confusion_matrix(Y_test, pre))


In [ ]:
ds['Type'].value_counts()

# Pan Tompkin's algorithm

In [ ]:
def preprocessing(high):
    tot=len(high)
    ##--DERIVATIVE
    deri=[]
    high=[0]*2+high+[0]*2
    for i in range(2,tot+2):
        deri.append((2*high[i+1]-2*high[i-1]+high[i+2]-high[i-2])/8)
    high=high[2:-2]
#     print(len(deri),len(high))
#     plt.plot(range(l,h),deri[l:h],c='r',label='derivative')
#     plt.plot(range(l,h),high[l:h],c='b',label='high pass')
#     plt.legend()
#     plt.show()
    ##--SQUARE
    sq=[]
    for i in range(len(deri)):
        sq.append(deri[i]*deri[i])
#     plt.plot(range(l,h),sq[l:h],c='b',label='squared')
#     plt.show()
    ##--MOVING AVERAGE
    mv=[]
    n=30
    sq=[0]*n+sq+[0]*n
    for i in range(n,len(deri)+n):
        mv.append(sum(sq[i:i+n])/n)
    sq=sq[n:-n]
#     print(len(mv),len(mv))
#     plt.plot(range(l,h),sq[l:h],c='r',label='squared')
#     plt.plot(range(l,h),mv[l:h],c='b',label='moving average')
#     plt.legend()
#     plt.show()
    ##--THRESHOLD
    sps=360
    spk,npk,thresh,beat_count,prev=0,0,0,0,False
    thresh1=[]
    index_arr=[]
    for j,i in enumerate(mv):
        if i>thresh:
            spk=0.125*i+0.875*spk
        else:
            npk=0.125*i+0.875*npk
        thresh=0.75*npk+0.25*spk
        thresh1.append(200*thresh)
        if i>thresh and prev==False:
            beat_count+=1
            prev=True
            index_arr.append(j)
        if i<thresh and prev==True:
            prev=False
#     print(len(thresh1))
#     plt.plot(range(l,h),high[l:h],c='b',label='Bandpass output')
#     plt.plot(range(l,h),thresh1[l:h],c='r',label='Threshold')
#     plt.legend()
#     plt.show()
    
    return beat_count,index_arr

# Testing the patients data with the trained model

In [ ]:
print(test_lst)

In [ ]:
def testing():
    dataset=pd.DataFrame(columns=['f{}'.format(i) for i in range(50)]+['Sample#'])
    for i in test_lst[:]:
        print('{} is processing'.format(i))
        df=pd.read_csv("../input/mitbih-database/mitbih_database/{}.csv".format(i))
        if i in [102,104]:
            df=df[["'sample #'","'V5'"]]
            df.rename(columns={"'V5'":"ML"},inplace=True)
        else:
            df=df[["'sample #'","'MLII'"]]
            df.rename(columns={"'MLII'":"ML"},inplace=True)
        df["ML"]=(df["ML"]-1024)*0.005
#         Filtering
        bandpass_output=filtering(df['ML'])
        print(df.head())
#         Preprocessing
        beat_count,index_arr=preprocessing(bandpass_output)
#         Interpolation
        total_peaks=interpolation(index_arr,bandpass_output)
#         Discrete Wavelet Transform
        CA,CD=wavelet_transform(total_peaks)
        temp_df=pd.DataFrame([i-j for i, j in zip(index_arr[1:],index_arr[:-1])] , columns=['Sample#'])
        DS=creating_dataframe(CA)
        DS=pd.concat([DS, temp_df] ,axis=1)
        print(CA.shape)
    return DS
X_test=testing()
type(X_test)

In [ ]:
y=clf2.predict(X_test)

In [ ]:
uni,cnt=np.unique(y,return_counts=True)

In [ ]:
uni

In [ ]:
cnt